In [82]:
import numpy as np
import cv2
import imutils
from imutils.perspective import four_point_transform
from imutils import contours

ANSWER_KEY = {0:1,1:4,2:0,3:3,4:1}

# Find Edges and Contours

In [108]:
omr_bgr = cv2.imread("images/omr_test_01.png")
#cv2.imshow("omr_bgr",omr_bgr)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
omr_gray = cv2.cvtColor(omr_bgr,cv2.COLOR_BGR2GRAY)
omr_blur = cv2.GaussianBlur(omr_gray,(5,5),0)
omr_edge = cv2.Canny(omr_blur,75,200)

cnts = cv2.findContours(omr_edge.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docContour = []
if(len(cnts) > 0):
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    for c in cnts:
        peri = cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c,0.02*peri,True)
        
        if(len(approx) == 4):
            docContour = approx
            break

# Apply Perspective Tranform

In [109]:
paper = four_point_transform(omr_bgr,docContour.reshape(4,2))
warped = four_point_transform(omr_gray,docContour.reshape(4,2))

# Otsu's Thresholding

In [118]:
_,thresh = cv2.threshold(warped,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
#kernel = np.ones((2,2),np.uint8)
#thresh = cv2.erode(thresh,kernel,iterations=2)
#thresh = cv2.dilate(thresh,kernel,iterations=7)
#thresh == cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
cv2.imshow("threshold",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find contours in threshold image

In [119]:
cnts = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

bubbleContours = []

for c in cnts:
    (x,y,w,h) = cv2.boundingRect(c)
    aspect_ratio =  w/float(h)
    
    if(w >= 20 and h >= 20 and aspect_ratio >= 0.9 and aspect_ratio <= 1.1):
        bubbleContours.append(c)
a = warped.copy()
cv2.drawContours(a,bubbleContours,-1,(255,0,0),2)
cv2.imshow("a",a)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Sort the contours top to bottom

In [136]:
bubbleContours = contours.sort_contours(bubbleContours,method = "top-to-bottom")[0]
correct = 0
a = paper.copy()
cv2.imshow("a",a)
cv2.waitKey(0)
cv2.destroyAllWindows()
for (q,i) in enumerate(np.arange(0,len(bubbleContours),5)):
    cnts = contours.sort_contours(bubbleContours[i:i+5])[0]
    bubbled = None
    for (j,c) in enumerate(cnts):
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask,[c],-1,255,-1)
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        cv2.imshow("a",mask)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        if bubbled is None or total > bubbled[0]:
            bubbled = (total,j)
    color = (0,0,255)
    k = ANSWER_KEY[q]
    if(k == bubbled[1]):
        color = (0,255,0)
        correct += 1
    cv2.drawContours(a,[cnts[k]],-1,color,3)
    cv2.imshow("a",a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [139]:
score = (correct / 5.0) *100
cv2.putText(paper, "{:.2f}%".format(score),(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,0,255),2)
cv2.imshow("exam",paper)
cv2.waitKey(0)
cv2.destroyAllWindows()